# Neural network working as a decoder (decodes a binary sequence to the number)
## Example: [1,0,0,0,1,0,0,1] -> 137

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## Create the dataset
- input: lists of size 8 with 0 and 1
- output: the value of the number

In [2]:
samples = []
labels = []
for x in range(256):
    bin_val = bin(x)[2:].rjust(8,'0')
    samples.append(list(bin_val))
    labels.append(x)
samples = np.array(samples,dtype=float)
labels = np.array(labels,dtype=float)

for i in range(5):
    print(samples[i],' -> ',labels[i])    
print('...')
for i in range(250,256):
    print(samples[i],' -> ',labels[i])

[0. 0. 0. 0. 0. 0. 0. 0.]  ->  0.0
[0. 0. 0. 0. 0. 0. 0. 1.]  ->  1.0
[0. 0. 0. 0. 0. 0. 1. 0.]  ->  2.0
[0. 0. 0. 0. 0. 0. 1. 1.]  ->  3.0
[0. 0. 0. 0. 0. 1. 0. 0.]  ->  4.0
...
[1. 1. 1. 1. 1. 0. 1. 0.]  ->  250.0
[1. 1. 1. 1. 1. 0. 1. 1.]  ->  251.0
[1. 1. 1. 1. 1. 1. 0. 0.]  ->  252.0
[1. 1. 1. 1. 1. 1. 0. 1.]  ->  253.0
[1. 1. 1. 1. 1. 1. 1. 0.]  ->  254.0
[1. 1. 1. 1. 1. 1. 1. 1.]  ->  255.0


## Build the neural network model

In [3]:
ADDITIONAL_LAYER = True

model = Sequential()
model.add(Dense(12,input_dim=8))
if ADDITIONAL_LAYER:
    model.add(Dense(12))
model.add(Dense(1))
model.summary()
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [4]:
model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae'])

## Divide to training and test 

In [5]:
TRAIN_SIZE = 0.1 # only 10% to train!

import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels, 
                            train_size=TRAIN_SIZE)
print('Training samples:',len(trainSamples),' test samples:',len(testSamples))

Training samples: 25  test samples: 231


## Train the network

In [14]:
EPOCHS = 200
print("Training using ",len(trainSamples),'samples and ',EPOCHS,' epochs')

H = model.fit(trainSamples,trainLabels,validation_data=(testSamples,testLabels), epochs=EPOCHS, verbose=0)
num_epochs+=EPOCHS
print('Epochs',num_epochs)
print("Initial loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][0],H.history['val_mae'][0]))
print("Final loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][-1],H.history['val_mae'][-1]))

Training using  25 samples and  200  epochs
Epochs 1200
Initial loss - training mae= 1.705 validation mae= 5.620
Final loss - training mae= 0.141 validation mae= 0.238


## Calculate errors

In [15]:
def calc_errors(predictedLabels,realLabels,verbose=1):
    predictedLabels = np.ravel(predictedLabels)
    for i in range(len(realLabels)):
        tNum = realLabels[i]
        pNum = predictedLabels[i]
        if(verbose>0):
            print("{:3.0f} -> {:6.2f} error={:7.2f}".format(tNum,pNum,abs(pNum-tNum)))
    print('Mean Absolute Error (MAE)= {:7.2f}'.format(np.mean(np.absolute(predictedLabels - realLabels))))
print('TRAINING SET')
predictedLabels = model.predict(trainSamples)
calc_errors(predictedLabels,trainLabels)
print()

print('TEST SET')
predictedLabels = model.predict(testSamples)
calc_errors(predictedLabels,testLabels)


TRAINING SET
160 -> 159.72 error=   0.28
208 -> 207.90 error=   0.10
 85 ->  84.87 error=   0.13
215 -> 214.78 error=   0.22
219 -> 219.00 error=   0.00
120 -> 119.88 error=   0.12
221 -> 220.83 error=   0.17
133 -> 132.87 error=   0.13
  2 ->   2.43 error=   0.43
196 -> 195.99 error=   0.01
 98 ->  97.98 error=   0.02
 96 ->  95.87 error=   0.13
123 -> 122.82 error=   0.18
129 -> 128.94 error=   0.06
 32 ->  31.93 error=   0.07
 99 ->  98.82 error=   0.18
 44 ->  44.02 error=   0.02
 69 ->  69.03 error=   0.03
186 -> 185.84 error=   0.16
232 -> 231.82 error=   0.18
157 -> 156.88 error=   0.12
127 -> 126.76 error=   0.24
166 -> 165.77 error=   0.23
250 -> 249.78 error=   0.22
 23 ->  23.04 error=   0.04
Mean Absolute Error (MAE)=    0.14

TEST SET
108 -> 107.96 error=   0.04
 88 ->  88.27 error=   0.27
 50 ->  49.89 error=   0.11
183 -> 182.45 error=   0.55
244 -> 243.45 error=   0.55
252 -> 251.61 error=   0.39
137 -> 137.10 error=   0.10
100 ->  99.81 error=   0.19
194 -> 194.17 erro